In [1]:
########################################
import os
import re
import json
import random
import pickle
import operator
import itertools

import numpy
import sklearn.ensemble
import sklearn.linear_model
import sklearn.feature_extraction
import pandas
import html
#import raha
########################################

In [2]:
import os
import re
import io
import sys
import math
import json
import html
import pickle
import difflib
import unicodedata
import bs4
import bz2
import py7zr
import numpy
import mwparserfromhell
import libarchive.public
#import dataset
import wikitextparser as wtp
from pprint import pprint
from wikitextparser import remove_markup, parse
import datetime
from pandas import DataFrame
import random
import pickle
import pandas as pd
import logging
from datasets import *
import pandas

In [3]:
import fasttext

In [4]:
#help(fasttext.FastText)

In [5]:
def evaluate_model (total_error, total_error_to_repair, total_correction):
        if total_error_to_repair==0:
            precision=0.00
        else:
            precision=total_correction/total_error_to_repair
            precision=round(precision,2)
        if total_error==0:
            recall=0.00
        else:
            recall=total_correction/total_error
            recall=round(recall,2)
        if (precision+recall)==0:
            f_score=0.00
        else:
            f_score=(2 * precision * recall) / (precision + recall)
            f_score=int((f_score * 100)) / 100.0
            #f_score=round(f_score,2)    
        return precision, recall,f_score

In [6]:
def value_normalizer(value):
        """
        This method takes a value and minimally normalizes it.
        """
        value = html.unescape(value)
        value = re.sub("[\t\n ]+", " ", value, re.UNICODE)
        value = value.strip("\t\n ")
        return value

In [7]:
def read_csv_dataset(dataset_path):
        """
        This method reads a dataset from a csv file path.
        """
        dataframe = pandas.read_csv(dataset_path, sep=",", header="infer", encoding="utf-8", dtype=str,
                                    keep_default_na=False, low_memory=False).applymap(value_normalizer)
        return dataframe


In [8]:
dataset_name = "tax"
dataset_dictionary = {
        "name": dataset_name,
        "path": os.path.abspath(os.path.join( "datasets", dataset_name, "dirty.csv")),
        "clean_path": os.path.abspath(os.path.join("datasets", dataset_name, "clean.csv"))
    }

In [9]:
name = dataset_dictionary["name"]
path = dataset_dictionary["path"]
dataframe = read_csv_dataset(dataset_dictionary["path"])
if "clean_path" in dataset_dictionary:
    has_ground_truth = True
    clean_path = dataset_dictionary["clean_path"]
    clean_dataframe = read_csv_dataset(dataset_dictionary["clean_path"])
if "repaired_path" in dataset_dictionary:
    has_been_repaired = True
    repaired_path = dataset_dictionary["repaired_path"]
    repaired_dataframe = read_csv_dataset(dataset_dictionary["repaired_path"])

In [10]:
dataframe.columns.values[0]

'f_name'

In [11]:
retrain_data=prepare_dataset_for_retrain_realworld(clean_dataframe, dataframe)

In [12]:
total_error=calculate_total_error_realworld(clean_dataframe, dataframe)

In [13]:
total_error

121219

# Flights

In [38]:
total_error_to_repaired=0
total_repaired=0
columns=list(dataframe.columns.values)
if columns[0]=="tuple_id":
    columns.remove(columns[0])
print(columns)  
for fds in columns:
    feature_data=columns
    target_col=fds
    #feature_data.remove(target_col)
    train_data_rows=[]
    data_for_retrain=retrain_data[feature_data]
    data_for_retrain[fds]='__label__' + data_for_retrain[fds].astype(str)
    data_for_retrain[fds]=[s.replace(" ", "_") for s in data_for_retrain[fds]]
    df3 = data_for_retrain[data_for_retrain[fds] != "__label__None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/training_flight_full.txt', index=False, sep=' ', header=False)
    ! wc datasets/training_flight_full.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/training_flight_full.txt",epoch=100)
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        #predict_value=retrain_data.at[index, fds[0]]
        predict_error=retrain_data.iloc[index]
        row=list(filter(None, predict_error))
        if row:
            row=row[1:]
            row = ' '.join([str(r) for r in row])
        
        #print(row)
            
       # total_error_to_repaired=total_error_to_repaired+1     
        repaired=model_flight.predict(row,k=1,threshold=0.991)
        print(repaired)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                #print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except:
            print("Threshold")
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)


['src', 'flight', 'sched_dep_time', 'act_dep_time', 'sched_arr_time', 'act_arr_time']
  2376  13920 144831 datasets/training_flight_full.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


  2376  13920 144831 datasets/training_flight_full.txt
 1465  9427 99014 datasets/training_flight_full.txt
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
T

((), array([], dtype=float64))
Threshold
(('__label__7:35_a.m.',), array([0.99402559]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__10:40_a.m.',), array([0.99396932]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(()

  818  5964 59786 datasets/training_flight_full.txt


...

/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], d

Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__5:26_p.m.',), array([0.99107218]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__12:09_p.m.',), array([0.99354255]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Thr

Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__3:04_p.m.',), array([0.99506509]))
((), array([], dtype=float64))
Threshold
(('__label__6:34_p.m.',), array([0.99838328]))
((), array([], dtype=float64))
Threshold
(('__label__7:36_p.m.',), array([0.99136138]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__5:26_p.m.',), array([0.99310124]))
(('__label__3:21_p.m.',), array([0.99393165]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__1:19_

Threshold
((), array([], dtype=float64))
Threshold
(('__label__9:56_a.m.',), array([0.99862587]))
((), array([], dtype=float64))
Threshold
(('__label__1:19_p.m.',), array([0.99903452]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__5:26_a.m.',), array([0.99633336]))
(('__label__7:17_a.m.',), array([0.99862611]))
((), array([], dtype=float64))
Threshold
(('__label__11:21_a.m.',), array([0.99493873]))
(('__label__3:31_p.m.',), array([0.99882126]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__9:43_a.m.',), array([0.99795896]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__10:29_p.m.',), array([0.99428421]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Thresho

# Hospital

In [55]:
total_error_to_repaired=0
total_repaired=0
columns=list(dataframe.columns.values)
if columns[0]=="tuple_id" or columns[0]=="index" :
    columns.remove(columns[0])
print(columns)  
for fds in columns:
    feature_data=columns
    target_col=fds
    #feature_data.remove(target_col)
    train_data_rows=[]
    data_for_retrain=retrain_data[feature_data]
    data_for_retrain[fds]='__label__' + data_for_retrain[fds].astype(str)
    data_for_retrain[fds]=[s.replace(" ", "_") for s in data_for_retrain[fds]]
    df3 = data_for_retrain[data_for_retrain[fds] != "__label__None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/training_flight_full.txt', index=False, sep=' ', header=False)
    ! wc datasets/training_flight_full.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/training_flight_full.txt",epoch=100)
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        #predict_value=retrain_data.at[index, fds[0]]
        predict_error=retrain_data.iloc[index]
        row=list(filter(None, predict_error))
        if row:
            row=row[1:]
            row = ' '.join([str(r) for r in row])
        
        #print(row)
            
       # total_error_to_repaired=total_error_to_repaired+1     
        repaired=model_flight.predict(row,k=1,threshold=0.9994)
        print(repaired)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                #print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except:
            print("Threshold")
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)


['provider_number', 'name', 'address_1', 'address_2', 'address_3', 'city', 'state', 'zip', 'county', 'phone', 'type', 'owner', 'emergency_service', 'condition', 'measure_code', 'measure_name', 'score', 'sample', 'state_average']
   972  40757 303863 datasets/training_flight_full.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], d

/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
   969  38091 30

...

((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
   969  39712 301074 datasets/training_flight_full.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
(('__label__0_patients',), array([0.99985945]))
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), arra

/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], dtype=float64))
Threshold
((), array([], d

# Tax

In [17]:
total_error_to_repaired=0
total_repaired=0
columns=list(dataframe.columns.values)
if columns[0]=="tuple_id" or columns[0]=="index" :
    columns.remove(columns[0])
#columns=['state','area_code','city']
print(columns)  
for fds in columns:
    if fds=="rate" or fds=="zip" or fds=="salary" or fds=="area_code":
        continue
    feature_data=columns
    target_col=fds
    #feature_data.remove(target_col)
    train_data_rows=[]
    data_for_retrain=retrain_data[feature_data]
    data_for_retrain[fds]='__label__' + data_for_retrain[fds].astype(str)
    data_for_retrain[fds]=[s.replace(" ", "_") for s in data_for_retrain[fds]]
    df3 = data_for_retrain[data_for_retrain[fds] != "__label__None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/training_flight_full.txt', index=False, sep=' ', header=False)
    ! wc datasets/training_flight_full.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/training_flight_full.txt",epoch=5,loss='hs')
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    #predict_data=retrain_data[feature_data]
    
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        #predict_value=retrain_data.at[index, fds[0]]
        predict_error=retrain_data.iloc[index]
        row=list(filter(None, predict_error))
        if row:
            #row=row[1:]
            row = ' '.join([str(r) for r in row])
        
       # print(row)
            
       # total_error_to_repaired=total_error_to_repaired+1     
        repaired=model_flight.predict(row,k=1,threshold=0.95)
        #print(repaired)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except:
            print("Threshold")
    print(total_error,total_error_to_repaired,total_repaired )
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)

['f_name', 'l_name', 'gender', 'area_code', 'phone', 'city', 'state', 'zip', 'marital_status', 'has_child', 'salary', 'rate', 'single_exemp', 'married_exemp', 'child_exemp']
  199728  2924610 16072191 datasets/training_flight_full.txt
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Thres

  200000  2928327 16092446 datasets/training_flight_full.txt
121219 0 0
  200000  2928327 16092446 datasets/training_flight_full.txt
121219 0 0
  199800  2876074 15986271 datasets/training_flight_full.txt
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Thres

  200000  2928327 16092446 datasets/training_flight_full.txt
121219 1199 1199
  199800  2925666 16077590 datasets/training_flight_full.txt
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
121219 1399 1399
121219 1399 1399
1.0 0.01 0.01


In [ ]:
ef transform_instance(row):
    cur_row = []
    #Prefix the index-ed label with __label__
    label = "__label__" + row[0]  
    cur_row.append(label)
    #Clean tweet and tokenize it
    cur_row.extend( nltk.word_tokenize(tweet_cleaning_for_sentiment_analysis(row[1].lower())))
def preprocess(input_file, output_file, keep=1):
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        with open(input_file, 'r', newline='') as csvinfile: #,encoding='latin1'
            csv_reader = csv.reader(csvinfile, delimiter=',', quotechar='"')
            for row in csv_reader:
                if row[0].upper() in ['POSITIVE','NEGATIVE','NEUTRAL','MIXED']:
                    row_output = transform_instance(row)
                    csv_writer.writerow(row_output)
# Preparing the training dataset 
preprocess('BetSentimentTweetAnalyzed.csv', 'tweets.train')

In [16]:
row

'NE 308'

In [ ]:
predict_data=retrain_data[feature_data]
for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        #predict_value=retrain_data.at[index, fds[0]]
        predict_error=predict_data.iloc[index]
        row=list(filter(None, predict_error))
        if row:
            row=row[1:]
            row = ' '.join([str(r) for r in row])
        
        print(row)
            
       # total_error_to_repaired=total_error_to_repaired+1     
        repaired=model_flight.predict(row,k=1,threshold=0.10)
        print(repaired)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                #print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except:
            print("Threshold")
    print(total_error,total_error_to_repaired,total_repaired )
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)

In [19]:
total_error_to_repaired

3569

In [20]:
total_repaired

0